<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0a4acde3419d47f9412085689dfd553e9f52d9bac491292db548585e4af3312f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 14:06:38
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -2.09 L (-1.49%)
Current PnL: -23.31 L (-15.53%)
CY Booked + Current PnL: -11.41 L (-7.6%)
-------------------
Total profit:  1.09 L
Total loss:  -24.40 L
-------------------
Total Booked + Current PnL: 15.57 L (12.65%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.61%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.51 L (65.5%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.29,54.0,M-SC,1.26,85451.0,-15326.0,15415.0,2.34,-15.21,18.04,0.09,245.0,-0.99,0.62,12.26,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.81,52.0,X-LC,3.72,211254.0,6754.0,16541.0,-1.38,3.30,7.83,11.39,10.0,0.41,1.54,12.55,X40N,ATH,PHARMA
83,VOLTAS,1530.00,-0.80,52.0,X-MC,1.02,208110.0,16368.0,21394.0,0.65,8.54,10.28,19.69,99.0,0.77,1.52,15.91,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.85,214500.0,4134.0,24646.0,0.00,1.97,11.49,13.68,37.0,0.17,1.56,18.81,XY25,NTT,REFINERIES
41,ITC,452.00,-40.03,39.0,X-LC,2.75,194304.0,-5834.0,26717.0,-0.48,-2.91,13.75,10.44,4.0,-0.22,1.42,2.49,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-36.03,65.0,M-SC,12.83,79651.0,-21508.0,96402.0,4.45,-21.26,121.03,74.03,202.0,-0.22,0.58,12.32,AR,BTT,AUTO
76,TTKPRESTIG,770.0,98.29,54.0,M-SC,1.26,85451.0,-15326.0,15415.0,2.34,-15.21,18.04,0.09,245.0,-0.99,0.62,12.26,OX40N,NTT,DURABLES
4,ANGELONE,3033.0,10.20,63.0,X-SC,10.33,195381.0,4375.0,53319.0,1.79,2.29,27.29,30.21,157.0,0.08,1.43,23.65,X40N,NTT,FINANCE
85,WIPRO,420.0,-12.43,54.0,M-LC,5.94,154064.0,3119.0,106335.0,1.37,2.07,69.02,72.51,53.0,0.03,1.12,8.21,XR,NTT,IT
17,CERA,9475.0,-23.15,44.0,H-SC,1.97,140174.0,-35729.0,77755.0,1.37,-20.31,55.47,23.89,149.0,-0.46,1.02,21.06,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-53.83,12.0,X-LC,1.06,145545.0,-130978.0,241052.0,-39.32,-47.37,165.62,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-278.42,23.0,H-SC,3.10,99280.0,-64325.0,185644.0,-4.42,-39.32,186.99,74.15,139.0,-0.35,0.72,27.26,XR,NTT,CHEMICALS
23,DEN,75.0,89.12,37.0,M-SC,3.87,52960.0,-31504.0,67042.0,-3.83,-37.30,126.59,42.07,237.0,-0.47,0.39,12.47,AR,NTT,ENTERTAINMENT
10,BANDHANBNK,400.0,-13.31,47.0,H-SC,2.96,217254.0,-61305.0,318755.0,-3.77,-22.01,146.72,92.42,151.0,-0.19,1.58,25.56,XY24,NTT,BANKS
68,SURYODAY,240.0,57.89,54.0,H-SC,10.46,145442.0,-33629.0,99351.0,-3.25,-18.78,68.31,36.71,135.0,-0.34,1.06,44.31,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123516.0,180.0,131446.0,-2.84,0.15,106.42,106.72,119.0,0.00,0.90,23.56,AR,ATH,MISC
54,RECLTD,446.00,45.36,44.0,M-LC,6.20,203968.0,798.0,41324.0,-0.07,0.39,20.26,20.74,55.0,0.02,1.49,5.96,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,56.0,H-SC,11.54,129060.0,-13419.0,32936.0,-1.93,-9.42,25.52,13.70,163.0,-0.41,0.94,52.33,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.29,54.0,M-SC,1.26,85451.0,-15326.0,15415.0,2.34,-15.21,18.04,0.09,245.0,-0.99,0.62,12.26,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220860.0,-48807.0,85142.0,-0.12,-18.10,38.55,13.47,138.0,-0.57,1.61,12.30,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,44.0,H-SC,1.97,140174.0,-35729.0,77755.0,1.37,-20.31,55.47,23.89,149.0,-0.46,1.02,21.06,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.05,42.0,H-MC,7.02,101470.0,-30365.0,72592.0,-0.21,-23.03,71.54,32.03,98.0,-0.42,0.74,13.65,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,44.0,M-LC,6.20,203968.0,798.0,41324.0,-0.07,0.39,20.26,20.74,55.0,0.02,1.49,5.96,XY25,NTT,FINANCE
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,220110.0,15034.0,97531.0,0.09,7.33,44.31,54.89,52.0,0.15,1.61,33.19,X5K,ATH,METALS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154064.0,3119.0,106335.0,1.37,2.07,69.02,72.51,53.0,0.03,1.12,8.21,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,52.0,M-LC,9.51,156167.0,15327.0,29156.0,-1.82,10.88,18.67,31.59,66.0,0.53,1.14,40.25,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.65,190774.0,10966.0,101454.0,-2.22,6.10,53.18,62.52,88.0,0.11,1.39,42.26,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,220110.0,15034.0,97531.0,0.09,7.33,44.31,54.89,52.0,0.15,1.61,33.19,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.65,190774.0,10966.0,101454.0,-2.22,6.10,53.18,62.52,88.0,0.11,1.39,42.26,XR,NTT,BANKS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154064.0,3119.0,106335.0,1.37,2.07,69.02,72.51,53.0,0.03,1.12,8.21,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123516.0,180.0,131446.0,-2.84,0.15,106.42,106.72,119.0,0.00,0.90,23.56,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.74,30.0,M-SC,3.41,86572.0,-4926.0,83680.0,-0.85,-5.38,96.66,86.07,223.0,-0.06,0.63,25.72,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,12.0,X-LC,1.06,145545.0,-130978.0,241052.0,-39.32,-47.37,165.62,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175972.0,-24534.0,121685.0,-1.31,-12.24,69.15,48.45,90.0,-0.20,1.28,1.48,X40N,ATH,FINANCE
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51312.0,-13694.0,164624.0,-1.76,-21.07,320.83,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226247.0,-21484.0,96540.0,-1.55,-8.67,42.67,30.30,92.0,-0.22,1.65,3.47,X40,ATH,ELECTRICAL
5,ASIANPAINT,3465.66,-19.07,31.0,X-LC,5.31,204398.0,-47370.0,100584.0,-0.85,-18.81,49.21,21.13,27.0,-0.47,1.49,9.94,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.68,42.0,H-LC,0.92,154700.0,-31395.0,78882.0,0.54,-16.87,50.99,25.51,15.0,-0.40,1.13,14.00,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.80,52.0,X-MC,1.02,208110.0,16368.0,21394.0,0.65,8.54,10.28,19.69,99.0,0.77,1.52,15.91,XY25,NTT,AC
72,TATAMOTORS,1065.00,-53.83,12.0,X-LC,1.06,145545.0,-130978.0,241052.0,-39.32,-47.37,165.62,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226247.0,-21484.0,96540.0,-1.55,-8.67,42.67,30.30,92.0,-0.22,1.65,3.47,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.58,41.0,X-MC,1.46,136471.0,338.0,30788.0,0.59,0.25,22.56,22.86,91.0,0.01,1.00,12.89,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51312.0,-13694.0,164624.0,-1.76,-21.07,320.83,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,12.0,X-LC,1.06,145545.0,-130978.0,241052.0,-39.32,-47.37,165.62,39.81,54.0,-0.54,1.06,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175972.0,-24534.0,121685.0,-1.31,-12.24,69.15,48.45,90.0,-0.20,1.28,1.48,X40N,ATH,FINANCE
82,VBL,671.64,-21.85,37.0,X-LC,10.51,281677.0,-34165.0,146838.0,-0.21,-10.82,52.13,35.67,5.0,-0.23,2.05,1.54,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.64,35.0,X-MC,7.68,220230.0,-43135.0,156473.0,-0.85,-16.38,71.05,43.04,84.0,-0.28,1.61,2.24,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51312.0,-13694.0,164624.0,-1.76,-21.07,320.83,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.14,34.0,X-SC,3.33,80304.0,-64856.0,140781.0,-1.11,-44.68,175.31,52.31,136.0,-0.46,0.59,7.49,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.72,33.0,X-SC,4.36,93109.0,-35561.0,80856.0,-2.37,-27.64,86.84,35.20,219.0,-0.44,0.68,6.87,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.71,44.0,X-SC,1.05,106114.0,4110.0,39814.0,-2.03,4.03,37.52,43.06,122.0,0.10,0.77,21.84,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.23,108120.0,-19818.0,66948.0,-0.32,-15.49,61.92,36.84,143.0,-0.30,0.79,10.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.97,43.0,X-LC,12.30,278588.0,-67368.0,135617.0,-1.45,-19.47,48.68,19.73,1.0,-0.50,2.03,2.61,X40,ATH,IT
39,INFY,2275.00,-18.16,52.0,X-LC,7.50,318456.0,5568.0,166107.0,0.13,1.78,52.16,54.87,3.0,0.03,2.32,8.55,X40,BTT,IT
41,ITC,452.00,-40.03,39.0,X-LC,2.75,194304.0,-5834.0,26717.0,-0.48,-2.91,13.75,10.44,4.0,-0.22,1.42,2.49,X40,NTT,FMCG
82,VBL,671.64,-21.85,37.0,X-LC,10.51,281677.0,-34165.0,146838.0,-0.21,-10.82,52.13,35.67,5.0,-0.23,2.05,1.54,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,41.0,H-LC,7.29,245616.0,-16003.0,135212.0,-0.59,-6.12,55.05,45.57,7.0,-0.12,1.79,4.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.72,81062.0,-32487.0,72534.0,-0.87,-28.61,89.48,35.27,268.0,-0.45,0.59,98.40,XR,NTT,HOTELS
48,MASFIN,398.61,-21.86,37.0,H-SC,9.90,88845.0,-9135.0,30740.0,-1.89,-9.32,34.60,22.05,152.0,-0.30,0.65,29.14,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,56.0,H-SC,11.54,129060.0,-13419.0,32936.0,-1.93,-9.42,25.52,13.70,163.0,-0.41,0.94,52.33,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.46,145442.0,-33629.0,99351.0,-3.25,-18.78,68.31,36.71,135.0,-0.34,1.06,44.31,XR,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.45,147168.0,-2052.0,146226.0,0.68,-1.38,99.36,96.62,208.0,-0.01,1.07,59.27,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.45,147168.0,-2052.0,146226.0,0.68,-1.38,99.36,96.62,208.0,-0.01,1.07,59.27,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.46,145442.0,-33629.0,99351.0,-3.25,-18.78,68.31,36.71,135.0,-0.34,1.06,44.31,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,220110.0,15034.0,97531.0,0.09,7.33,44.31,54.89,52.0,0.15,1.61,33.19,X5K,ATH,METALS
50,PGHH,17907.65,-26.45,60.0,X-MC,2.91,213030.0,12210.0,55580.0,0.16,6.08,26.09,33.76,80.0,0.22,1.55,11.59,X40,ATH,FMCG
63,SHALBY,327.00,1163.16,52.0,M-SC,22.89,171760.0,-9383.0,54843.0,-2.75,-5.18,31.93,25.10,235.0,-0.17,1.25,37.65,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,44.00
2,50,75.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.40
LC    33.70
MC    21.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.66
X40      14.85
XY25     12.12
XR       10.96
X40N     10.09
AR        8.23
OX40N     7.70
X200      1.80
X5K       1.61
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.61
X-LC    22.59
X-MC    16.14
M-SC    12.72
M-LC     5.36
H-LC     4.72
X-SC     4.63
H-MC     3.80
M-MC     1.62
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.49,41.59
IT,13.06,-17.32,80.87
FINANCE,12.17,-13.29,64.65
MISC,6.91,-21.06,88.14
BANKS,6.42,-13.22,72.78
PAINTS,5.62,-19.91,37.78
ELECTRICAL,5.36,-12.07,51.39
INSURANCE,3.84,-5.34,44.72
AUTO,2.84,-47.40,117.87


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3369894.0,22
XR,1405974.0,14
AR,1200706.0,9
X40,803651.0,10
X40N,643989.0,8
OX40N,588927.0,10
XY25,522110.0,8
SR,268594.0,2
X5K,97531.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3487013.0,25
M-SC,1295270.0,15
X-LC,1142490.0,13
X-MC,1116498.0,11
X-SC,546342.0,6
H-MC,359169.0,3
H-LC,292858.0,3
M-LC,274346.0,4
L-SC,263329.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215592.0      6
           XR         831611.0      7
           AR         823168.0      5
M-SC       XY24       703093.0      6
X-MC       XY24       590950.0      4
X-LC       X40        496799.0      5
           XY24       310628.0      2
X-SC       X40N       300862.0      4
H-SC       OX40N      277200.0      4
           SR         268594.0      2
X-MC       X40        225996.0      4
X-LC       X40N       221442.0      3
H-LC       AR         214094.0      2
H-MC       XY24       185123.0      1
X-MC       XY25       177867.0      2
L-SC       XR         173175.0      2
M-MC       XY24       170728.0      1
X-SC       XY24       164624.0      1
M-SC       AR         163444.0      2
           OX40N      148981.0      4
           XY25       146226.0      1
           XR         133526.0      2
X-MC       X40N       121685.0      1
X-LC       XY25       113621.0      3
M-LC       XR         106335.0      1
H-MC       XR         101454.0      1
M-LC       X5K         97531.0      1
L-SC       OX40N       90154.0      1
X-SC       X40         80856.0      1
H-LC       X200        78764.0      1
H-MC       OX40N       72592.0      1
H-SC       MH          70848.0      1
L-MC       XR          59873.0      1
L-LC       XY25        43072.0      1
M-LC       XY25        41324.0      1
           XY24        29156.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
